## Install Dependency

In [ ]:
!pip install frida-tools

## Install Jadx

In [ ]:
import os
import urllib.request
import zipfile

from var import *

if not os.path.exists(jadx_dir):
    zip_file = f"{jadx_dir}.zip"
    urllib.request.urlretrieve(jadx_download_url, zip_file)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(jadx_dir)
    os.chmod(jadx_gui_bin, 0o744)
    os.remove(zip_file)

## Steup Frida

In [ ]:
import urllib.request
import os
import lzma
from var import *

if not os.path.exists(frida_server_bin_path):
    cpu_arch = !adb shell getprop ro.product.cpu.abi
    cpu_arch = cpu_arch[0].strip() if cpu_arch else ""

    # 아키텍처 매핑
    arch_map = {
        "armeabi-v7a": "arm",
        "arm64-v8a": "arm64",
        "x86": "x86",
        "x86_64": "x86_64"
    }

    frida_arch = arch_map.get(cpu_arch)
    frida_server_download_url = f"https://github.com/frida/frida/releases/download/{frida_version}/frida-server-{frida_version}-android-{frida_arch}.xz"

    os.makedirs("utils", exist_ok=True)

    download_path = f"{frida_server_bin_path}.xz"
    print(f"Downloading frida server from {frida_server_download_url}...")
    urllib.request.urlretrieve(frida_server_download_url, download_path)
    print(f"Downloaded to {download_path}")

    # 압축 풀기
    file_name = "frida-server"
    print(f"Extracting frida-server...")
    with lzma.open(download_path, "rb") as compressed_file:
        with open(frida_server_bin_path, "wb") as extracted_file:
            extracted_file.write(compressed_file.read())
    print(f"Extracted to {frida_server_bin_path}")

    # 다운로드된 압축 파일 삭제
    os.remove(download_path)
    print(f"Removed compressed file {download_path}")


## Setup Path for Android Studio Emulator

In [ ]:
from utils.utils import setup_android_path_env

setup_android_path_env()

In [ ]:
import subprocess
import re

def list_and_select_sorted_system_image():
    try:
        # sdkmanager --list 명령 실행
        result = subprocess.run(
            ["sdkmanager", "--list"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        if result.returncode != 0:
            print(f"Error: {result.stderr}")
            return

        # stdout에서 system-images만 필터링
        system_images = [
            line.strip() for line in result.stdout.splitlines()
            if line.strip().startswith("system-images;")
        ]

        if not system_images:
            print("No system images found.")
            return

        # 정렬: 숫자가 우선, 내림차순 -> 숫자가 아닌 항목은 마지막으로
        def sort_key(image):
            match = re.search(r"system-images;android-(\d+)", image)
            if match:
                return -int(match.group(1))  # 숫자를 음수로 변환해 내림차순 정렬
            return float("inf")  # 숫자가 아닌 경우 가장 뒤로

        sorted_images = sorted(system_images, key=sort_key)

        # 정렬된 시스템 이미지 출력
        print("Available System Images (sorted):")
        for idx, image in enumerate(sorted_images, start=1):
            print(f"{idx}. {image}")

        # 사용자로부터 선택 입력 받기
        while True:
            try:
                choice = int(input("Select a system image (enter the number): "))
                if 1 <= choice <= len(sorted_images):
                    selected_image = sorted_images[choice - 1]
                    print(f"You selected: {selected_image}")
                    return selected_image
                else:
                    print(f"Please select a number between 1 and {len(sorted_images)}.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    except FileNotFoundError:
        print("Error: sdkmanager command not found. Ensure Android SDK is installed and in PATH.")

## Run Jadx

In [ ]:
from utils.utils import run_command_in_background
from var import *

run_command_in_background(f"{jadx_gui_bin}")

## Run Emulator

In [ ]:
from utils.utils import run_command_in_background
from var import *

# AVD 확인 및 생성
avds = !emulator -list-avds
if avd_name not in avds:
    system_image = system_image if system_image else list_and_select_sorted_system_image()
    if system_image:
        !avdmanager create avd -n {avd_name} -k "{system_image}" --device "{device}" --force

# 실행 중인지 확인 후 에뮬레이터 실행
avds = !emulator -list-avds
if avd_name in avds:
    print(f"Starting emulator '{avd_name}'...")
    run_command_in_background(f'emulator -avd {avd_name}')
else:
    print(f"AVD '{avd_name}' not found.")


## Run frida server

In [ ]:
from utils.utils import run_command_in_background
from var import *

!adb root
frida = !adb shell ls /data/local/tmp/frida # 2>/dev/null
frida_exist = len(frida) != 0
if not frida_exist:
    !adb push {frida_server_bin_path} /data/local/tmp/frida
    !adb shell chmod +x /data/local/tmp/frida

run_command_in_background("adb shell /data/local/tmp/frida -D")

## Install apk

In [ ]:
apk_path = ""
!adb install {apk_path}

In [ ]:
import frida
device = frida.get_usb_device()

In [ ]:
from utils.utils import get_focused_app_package
from var import package_name

package_name = package_name if package_name else get_focused_app_package()

In [ ]:
from var import package_name, frida_script_path

# 타겟 앱을 실행합니다.
pid = device.spawn([package_name])
session = device.attach(pid)

if os.path.exists(frida_script_path) and os.path.isfile(frida_script_path):
    # Frida JavaScript 스크립트를 로드합니다.
    with open(frida_script_path, 'r', encoding='utf-8') as f:
        frida_code = f.read()
    frida_script_path = session.create_script(frida_code)
    frida_script_path.load()
device.resume(pid)

In [ ]:
frida_script_path.unload()